In [ ]:
import pandas as pd
import numpy as np
import random
from scipy.integrate import odeint
from scipy.optimize import curve_fit
from lmfit import minimize, Parameters, Parameter, report_fit
import matplotlib.pyplot as plt
import os
#==============================================================================
''' Make new data matrix, same as csv except infected cells are one total for convience '''
DataFrame = pd.read_csv('Cell_count_data_Tromas_2014.csv') # Read data from file

TROMAS_DATA = np.empty((DataFrame.shape[0], 5), int)
for i in range(DataFrame.shape[0]): # Number of rows
    for j in range(5):              # Number of columns desired
        TROMAS_DATA[i][j] = DataFrame.iloc[i, j]
        if j == 4:
            TROMAS_DATA[i][j] = DataFrame.ix[i, 'Venus_only'] + DataFrame.ix[i, 'BFP_only'] + DataFrame.ix[i, 'Mixed']
'''
Col 0: Days post infection
Col 1: Leaf number
Col 2: Replicate plant number
Col 3: Number of unifected cells
Col 4: Number of total infected cells
'''
#==============================================================================
''' Create subplot array '''
fig, ax = plt.subplots(2, 2, figsize = (8, 8))
fig.subplots_adjust(hspace = .25, wspace = .2)
#==============================================================================
''' Make axis for data points '''
DAYS_AXIS = [3, 5, 7, 10]
#==============================================================================
''' Make axis for negative log likelihood '''
ZERO_DAYS_AXIS = [0, 3, 5, 7, 10]
#==============================================================================
''' Time axis for models '''
t = np.linspace(0, 10, 1000)
#==============================================================================
''' Time axis for models '''
t3 = np.linspace(3, 10, 1000)
#==============================================================================
''' Parameter list (Initialized with already predicted values) '''
parameters = Parameters()
parameters.add('b', value = .865, min = .0001, max = 1.0000)
parameters.add('x5', value = .030, min = .0001, max = 2.0000)
parameters.add('x6', value = .214, min = .0001, max = 2.0000)
parameters.add('x7', value = .016, min = .0001, max = 2.0000)
parameters.add('psi3', value = .057, min = .0001, max = 1.0000)
parameters.add('psi5', value = .016, min = .0001, max = 1.0000)
parameters.add('psi6', value = .235, min = .0001, max = 1.0000)
parameters.add('psi7', value = .283, min = .0001, max = 1.0000)
#==============================================================================
'''  Initial conditions '''
I0 = 3.72 * 10**-3
Ik0 = [I0, 0, 0, 0]
#==============================================================================
''' Define the model, Eq. (1) on pg. 3 '''
def original(Ik, t):
    I0 = 3.72 * 10**-3  # Original 3.72 * 10**-3
    b = .865            # Original .871
    x5 = .030           # Original .724
    x6 = .214           # Original 1.38
    x7 = .016           # Original .107
    psi3 = .057         # Original .083
    psi5 = .016         # Original .018
    psi6 = .235         # Original .233
    psi7 = .283         # Original .286

    I3 = Ik[0]
    I5 = Ik[1]
    I6 = Ik[2]
    I7 = Ik[3]

    if (I3 < psi3):
        S3 = (1 - (I3 / psi3))
    else:
        S3 = 0
    if (I5 < psi5):
        S5 = (1 - (I5 / psi5))
    else:
        S5 = 0
    if (I6 < psi6):
        S6 = (1 - (I6 / psi6))
    else:
        S6 = 0
    if (I7 < psi7):
        S7 = (1 - (I7 / psi7))
    else:
        S7 = 0

    dI3dt = b * I3 * S3
    dI5dt = b * I5 * S5 + x5 * S5 * I3
    dI6dt = b * I6 * S6 + x6 * S6 * (I3 + I5)
    dI7dt = b * I7 * S7 + x7 * S7 * (I3 + I5 + I6)

    return [dI3dt, dI5dt, dI6dt, dI7dt]
#==============================================================================
def model(Mk, t, parameters):
    M3 = Mk[0]
    M5 = Mk[1]
    M6 = Mk[2]
    M7 = Mk[3]

    try: # Get parameters
        b = parameters['b'].value
        x5 = parameters['x5'].value
        x6 = parameters['x6'].value
        x7 = parameters['x7'].value
        psi3 = parameters['psi3'].value
        psi5 = parameters['psi5'].value
        psi6 = parameters['psi6'].value
        psi7 = parameters['psi7'].value
    except KeyError:
        b, x5, x6, x7, psi3, psi5, psi6, psi7 = parameters

    if (M3 < psi3):
        S3 = (1 - (M3 / psi3))
    else:
        S3 = 0
    if (M5 < psi5):
        S5 = (1 - (M5 / psi5))
    else:
        S5 = 0
    if (M6 < psi6):
        S6 = (1 - (M6 / psi6))
    else:
        S6 = 0
    if (M7 < psi7):
        S7 = (1 - (M7 / psi7))
    else:
        S7 = 0

    dM3dt = b * M3 * S3
    dM5dt = b * M5 * S5 + x5 * S5 * M3
    dM6dt = b * M6 * S6 + x6 * S6 * (M3 + M5)
    dM7dt = b * M7 * S7 + x7 * S7 * (M3 + M5 + M6)

    return [dM3dt, dM5dt, dM6dt, dM7dt]
#==============================================================================
''' Add random noise to model prediction to try to recover parameters '''
def negLogLike(parameters):
    # Solve ODE system to get model values; parameters are not yet fitted
    MM = odeint(model, Ik0, ZERO_DAYS_AXIS, args=(parameters,))
    M3 = MM[:, 0]
    M5 = MM[:, 1]
    M6 = MM[:, 2]
    M7 = MM[:, 3]

    nll = 0;
    num_cells = 10000
    epsilon = 10**-10
    for i in range(len(NOISY_DATA) - 1):          # Iterate through all rows but skip first row
        for j in range(len(NOISY_DATA[0])):       # Iterate through all columns
            Vktp = NOISY_DATA[i + 1][j] * num_cells   # Number of infected cells
            Aktp = num_cells                  # Total number of cells observed
            Iktp = MM[i + 1][j]               # Frequency of cellular infection

            if (Iktp <= 0):
                Iktp = epsilon
            if (Iktp >= 1):
                Iktp = 1 - epsilon

            nll += Vktp * np.log(Iktp) + (Aktp - Vktp) * np.log(1 - Iktp)
    
    return [-nll]
#==============================================================================
''' Make data based on original model with random noise '''
NOISY_DATA = np.empty((len(ZERO_DAYS_AXIS), 4), float)

TROMAS_MODEL_DATA = odeint(original, Ik0, ZERO_DAYS_AXIS) # Make array of model predictions

for i in range(len(TROMAS_MODEL_DATA)):
    for j in range(len(TROMAS_MODEL_DATA[0])):
        NOISY_DATA[i][j] = TROMAS_MODEL_DATA[i][j] + TROMAS_MODEL_DATA[i][j] * random.uniform(-.1, .1) # Add proportional random noise
#==============================================================================
''' Miminize negative log likelihood with differential evolution algorithm '''
result_evo = minimize(negLogLike, parameters, method = 'differential_evolution')
#==============================================================================
''' Compare fitted values with those in Tromas' paper '''
print("======================================================================")
print("======================================================================")
report_fit(result_evo)
print("Original beta = .865, My beta = ", result_evo.params['b'].value)
print("Original x5   = .030, My x5   = ", result_evo.params['x5'].value)
print("Original x6   = .214, My x6   = ", result_evo.params['x6'].value)
print("Original x7   = .016, My x7   = ", result_evo.params['x7'].value)
print("Original psi3 = .057, My psi3 = ", result_evo.params['psi3'].value)
print("Original psi5 = .016, My psi5 = ", result_evo.params['psi5'].value)
print("Original psi6 = .235, My psi6 = ", result_evo.params['psi6'].value)
print("Original psi7 = .283, My psi7 = ", result_evo.params['psi7'].value)
print("======================================================================")
print("======================================================================")
#==============================================================================
''' Solve original system of differential equations'''
II = odeint(original, Ik0, t)
I3 = II[:, 0]
I5 = II[:, 1]
I6 = II[:, 2]
I7 = II[:, 3]
#==============================================================================
''' Solve ODE system with fitted model parameters '''
MM_EVO = odeint(model, Ik0, t, args=(result_evo.params,))
M3_EVO = MM_EVO[:, 0]
M5_EVO = MM_EVO[:, 1]
M6_EVO = MM_EVO[:, 2]
M7_EVO = MM_EVO[:, 3]
#==============================================================================
''' Plot experimental data and model curves '''
for leaf_iterator in range(4):
#==============================================================================
    ''' Set up matricies '''
    RATIOS = []
    LEAF_DATA = np.empty((5, 4), float)  # Set up matrix
    AVERAGES = []
    #==========================================================================
    ''' Compute data from csv '''
    for i in range(round(len(TROMAS_DATA) / 4)):  # Iterate through all rows
        non_infected = TROMAS_DATA[(4 * i + leaf_iterator)][3]
        infected = TROMAS_DATA[(4 * i + leaf_iterator)][4]
        ratio = infected / (non_infected + infected)
        RATIOS.append(ratio)
    #==========================================================================
    ''' Build a matrix of ratios with each row being a replicate '''
    for i in range(5):
        for j in range(4):
            LEAF_DATA[i][j] = RATIOS[i + 5 * j]
    #==========================================================================
    ''' Calculate average cellular infection per day '''
    for i in range(4):
        sum = 0
        for j in range(5):
            sum += LEAF_DATA[j][i]
            mean = sum / 5
        AVERAGES.append(mean)
    #==========================================================================
    ''' Plot results '''
    if leaf_iterator == 0:
        for i in range(5):
            ax[0][0].scatter(DAYS_AXIS, LEAF_DATA[i, :], s = 80, facecolors = 'none', edgecolors = 'b')
        ax[0][0].scatter(DAYS_AXIS, LEAF_DATA[0, :], s = 80, facecolors = 'none', edgecolors = 'b', label = "Plants") # Plot copy to get one label
        ax[0][0].plot(DAYS_AXIS, AVERAGES, 'r-', label = "Average", linewidth = 1)

        ax[0][0].plot(t, I3, color = 'black', label = "Fitted model on original data")   # Plot differential equation
        ax[0][0].plot(t, M3_EVO, color = 'green', label = "Fitted model on data with random noise")

        ax[0][0].legend(loc = "upper left")
        ax[0][0].set_xlim(0, 12)
        ax[0][0].set_ylim(0, .5)
        ax[0][0].set_xlabel('Days post innoculation')
        ax[0][0].set_ylabel('Frequency of celluar infection')
        ax[0][0].text(10.25, .4, 'Leaf 3')

    elif leaf_iterator == 1:
        for i in range(5):
            ax[0][1].scatter(DAYS_AXIS, LEAF_DATA[i, :], s = 80, facecolors = 'none', edgecolors = 'b')
        ax[0][1].scatter(DAYS_AXIS, LEAF_DATA[0, :], s = 80, facecolors = 'none', edgecolors = 'b', label = "Plants") # Plot copy to get one label
        ax[0][1].plot(DAYS_AXIS, AVERAGES, 'r-', label = "Average", linewidth = 1)

        ax[0][1].plot(t, I5, color = 'black', label = "Fitted model on original data")   # Plot differential equation
        ax[0][1].plot(t, M5_EVO, color = 'green', label = "Fitted model on data with random noise")

        ax[0][1].legend(loc = "upper left")
        ax[0][1].set_xlim(0, 12)
        ax[0][1].set_ylim(0, .5)
        ax[0][1].set_xlabel('Days post innoculation')
        ax[0][1].set_ylabel('Frequency of celluar infection')
        ax[0][1].text(10.25, .4, 'Leaf 5')

    elif leaf_iterator == 2:
        for i in range(5):
            ax[1][0].scatter(DAYS_AXIS, LEAF_DATA[i, :], s = 80, facecolors = 'none', edgecolors = 'b')
        ax[1][0].scatter(DAYS_AXIS, LEAF_DATA[0, :], s = 80, facecolors = 'none', edgecolors = 'b', label = "Plants") # Plot copy to get one label
        ax[1][0].plot(DAYS_AXIS, AVERAGES, 'r-', label = "Average", linewidth = 1)

        ax[1][0].plot(t, I6, color = 'black', label = "Fitted model on original data")   # Plot differential equation
        ax[1][0].plot(t, M6_EVO, color = 'green', label = "Fitted model on data with random noise")

        ax[1][0].legend(loc = "upper left")
        ax[1][0].set_xlim(0, 12)
        ax[1][0].set_ylim(0, .5)
        ax[1][0].set_xlabel('Days post innoculation')
        ax[1][0].set_ylabel('Frequency of celluar infection')
        ax[1][0].text(10.25, .4, 'Leaf 6')

    elif leaf_iterator == 3:
        for i in range(5):
            ax[1][1].scatter(DAYS_AXIS, LEAF_DATA[i, :], s = 80, facecolors = 'none', edgecolors = 'b')
        ax[1][1].scatter(DAYS_AXIS, LEAF_DATA[0, :], s = 80, facecolors = 'none', edgecolors = 'b', label = "Plants") # Plot copy to get one label
        ax[1][1].plot(DAYS_AXIS, AVERAGES, 'r-', label = "Average", linewidth = 1)

        ax[1][1].plot(t, I7, color = 'black', label = "Fitted Model on Tromas Data")   # Plot differential equation
        ax[1][1].plot(t, M7_EVO, color = 'green', label = "Fitted model on data with random noise")

        ax[1][1].legend(loc = "upper left")
        ax[1][1].set_xlim(0, 12)
        ax[1][1].set_ylim(0, .5)
        ax[1][1].set_xlabel('Days post innoculation')
        ax[1][1].set_ylabel('Frequency of celluar infection')
        ax[1][1].text(10.25, .4, 'Leaf 7')
#==============================================================================
plt.show()
plt.show()
#==============================================================================
''' Save figure '''
save_path = r'C:\Users\joshm\OneDrive\Documents\Plant_Gang\Figures_Images' # Path to where you want the file to be saved
filename = "TromasModelParametersEstimationv4.pdf" # Specify filename
fig.savefig(os.path.join(save_path, filename)) # Save figure in the new directory